## Electric Vehicle Data

The following analysis uses electric vehicle data that is linked to Google BigQuery database to understand some key metrics from electric car users charging habits

## Database structure
![Screenshot 2026-02-03 162242](Screenshot%202026-02-03%20162242.png)


### Questions to answer

1. What is the most common day and time for charging each month?
2. What is the longest charging duration per month?
3. What is the average charge rate per month?

In [10]:
-- Explore the data in the table
SELECT *
FROM VEHICLES.CHARGING_SESSIONS
LIMIT 5

,garage_id,user_id,user_type,start_plugin,start_plugin_hour,end_plugout,end_plugout_hour,el_kwh,duration_hours,month_plugin,weekdays_plugin
0,Bl2,Bl2-1,Private,2019-01-25 15:39:00+00:00,15,2019-01-25 15:52:00+00:00,15,0.71,0.204444,Jan,Friday
1,Bl2,Bl2-1,Private,2019-01-25 15:52:00+00:00,15,2019-01-25 15:52:00+00:00,15,0.01,0.007778,Jan,Friday
2,Bl2,Bl2-1,Private,2019-01-25 15:52:00+00:00,15,2019-01-25 16:01:00+00:00,16,0.47,0.143611,Jan,Friday
3,Bl2,Bl2-1,Private,2019-01-25 16:04:00+00:00,16,2019-01-25 16:05:00+00:00,16,0.03,0.013056,Jan,Friday
4,Bl2,Bl2-1,Private,2019-01-25 16:09:00+00:00,16,2019-01-25 16:50:00+00:00,16,2.39,0.669722,Jan,Friday


## Question 1: What is the most common day and time for charging each month?

### Approach

In order to find the answer to this question, we need to extract, aggregate and then rank the data for each month to understand where the highest (or most common) days and hours land for each month

In [11]:
WITH MonthlyCounts AS (
    SELECT 
        -- 1. Extract numeric month for chronological sorting
        EXTRACT(MONTH FROM start_plugin) AS month_num,
        -- 2. Format names for the final output. This has to be changed from TO_CHAR when working in BigQuery
        FORMAT_TIMESTAMP('%B', start_plugin) AS charge_month,
        FORMAT_TIMESTAMP('%A', start_plugin) AS day_of_week,
        start_plugin_hour AS charge_hour,
        COUNT(*) AS session_count
    FROM VEHICLES.CHARGING_SESSIONS 
    GROUP BY 1, 2, 3, 4
),
RankedSessions AS (
    SELECT 
        *,
        -- 3. ROW_NUMBER ensures exactly 1 row per month. Without this, there were multiple instances per month. This method approach will pick the first day/hour alphabetically/numerically.
        ROW_NUMBER() OVER (
            PARTITION BY month_num 
            ORDER BY session_count DESC, day_of_week ASC, charge_hour ASC
        ) as frequency_row
    FROM MonthlyCounts
)
-- 4. 
SELECT 
    charge_month,
    day_of_week,
    charge_hour,
    session_count
FROM RankedSessions
WHERE frequency_row = 1
ORDER BY month_num ASC;

,charge_month,day_of_week,charge_hour,session_count
0,January,Friday,16,49
1,February,Saturday,17,5
2,March,Saturday,17,10
3,April,Friday,16,9
4,May,Thursday,19,11
5,June,Friday,15,11
6,July,Wednesday,19,7
7,August,Friday,16,11
8,September,Friday,16,15
9,October,Thursday,16,25


Looking at the outputs of this, it appears that the most common or most frequent charging takes place in the evening for each month. This could be a result of the ranking function being used, so we will adjust the query to look at the values for the morning and evening, to see if this representation is actually true or being masked by the RANK function

In [13]:
WITH CategorisedCharges AS (
    SELECT 
        EXTRACT(MONTH FROM start_plugin) AS month_num,
        FORMAT_TIMESTAMP('%B', start_plugin) AS charge_month,
        FORMAT_TIMESTAMP('%A', start_plugin) AS day_of_week,
	-- This query has been added to split the data for each month into a morning peak and an evening peak. This will allow comparison between morning and evening peaks.
	CASE 
            WHEN EXTRACT(HOUR FROM start_plugin) < 12 THEN 'Morning Peak (00:00-11:59)'
            ELSE 'Evening Peak (12:00-23:59)'
        END AS peak_period,
        EXTRACT(HOUR FROM start_plugin) AS charge_hour,
        COUNT(*) AS session_count
    FROM `VEHICLES.CHARGING_SESSIONS`
    GROUP BY 1, 2, 3, 4, 5
),
RankedPeaks AS (
    SELECT 
        *,
        ROW_NUMBER() OVER (
            PARTITION BY month_num, peak_period 
            ORDER BY session_count DESC, charge_hour ASC
        ) as frequency_row
    FROM CategorisedCharges
)
SELECT 
    charge_month,
    peak_period,
    day_of_week,
    charge_hour,
    session_count
FROM RankedPeaks
WHERE frequency_row = 1
ORDER BY month_num ASC, peak_period DESC;

,charge_month,peak_period,day_of_week,charge_hour,session_count
0,January,Morning Peak (00:00-11:59),Wednesday,7,7
1,January,Evening Peak (12:00-23:59),Friday,16,49
2,February,Morning Peak (00:00-11:59),Wednesday,11,2
3,February,Evening Peak (12:00-23:59),Tuesday,16,5
4,March,Morning Peak (00:00-11:59),Saturday,0,3
5,March,Evening Peak (12:00-23:59),Saturday,17,10
6,April,Morning Peak (00:00-11:59),Wednesday,0,2
7,April,Evening Peak (12:00-23:59),Monday,16,9
8,May,Morning Peak (00:00-11:59),Friday,0,2
9,May,Evening Peak (12:00-23:59),Thursday,19,11


By charting the output, we can see that charging in the evenings is the most common result and was not being masked by the RANK function.

## 2. What is the longest charging duration per month?



In [14]:
WITH MonthlyDurations AS (
    SELECT 
        month_plugin,
        duration_hours,
        user_id,
        garage_id,
        start_plugin,
        -- Rank durations from longest to shortest per month
        ROW_NUMBER() OVER (
            PARTITION BY month_plugin 
            ORDER BY duration_hours DESC
        ) as length_rank
    FROM `VEHICLES.CHARGING_SESSIONS`
)
SELECT 
    month_plugin,
    duration_hours AS max_duration,
    user_id,
    garage_id,
    start_plugin
FROM MonthlyDurations
WHERE length_rank = 1
ORDER BY PARSE_DATE('%B', month_plugin);

,month_plugin,max_duration,user_id,garage_id,start_plugin
0,Jan,187.056944,AdO1-2,AdO1,2019-01-21 16:27:00+00:00
1,Feb,239.240278,Share-8,Bl2,2019-02-04 16:00:00+00:00
2,Mar,132.258333,AsO2-1,AsO2,2019-03-06 18:42:00+00:00
3,Apr,105.741389,Bl2-3,Bl2,2019-04-30 05:02:00+00:00
4,May,113.095833,Bl2-6,Bl2,2019-05-23 18:00:00+00:00
5,Jun,69.937500,AdO3-4,AdO3,2019-06-10 19:45:00+00:00
6,Jul,161.528333,AdO1-2,AdO1,2019-07-08 19:38:00+00:00
7,Aug,255.029444,Bl2-3,Bl2,2019-08-01 18:02:00+00:00
8,Sep,142.459167,AdO1-1,AdO1,2019-09-22 20:27:00+00:00
9,Oct,119.660000,MS1-1,MS1,2019-10-06 15:26:00+00:00


## 3. What is the average charge rate per month?



In [15]:
SELECT 
    month_plugin,
    -- Calculate average rate (kW = kWh / Hours)
    ROUND(AVG(el_kwh / NULLIF(duration_hours, 0)), 2) AS avg_rate_kw,
    -- Also helpful to see total energy vs total time
    ROUND(SUM(el_kwh) / SUM(NULLIF(duration_hours, 0)), 2) AS weighted_avg_rate_kw,
    COUNT(*) AS total_sessions
FROM `VEHICLES.CHARGING_SESSIONS`
WHERE duration_hours > 0 -- Exclude sessions with no duration
GROUP BY 1, EXTRACT(MONTH FROM start_plugin)
ORDER BY EXTRACT(MONTH FROM start_plugin) ASC;

,month_plugin,avg_rate_kw,weighted_avg_rate_kw,total_sessions
0,Jan,2.03,1.03,1360
1,Feb,1.95,0.74,135
2,Mar,1.91,1.01,301
3,Apr,1.97,0.99,306
4,May,2.42,1.12,288
5,Jun,2.51,1.34,291
6,Jul,2.61,1.42,238
7,Aug,2.64,1.44,362
8,Sep,2.54,1.47,501
9,Oct,2.19,1.17,845


In [16]:
SELECT 
    month_plugin,
    user_type,
    -- Calculate average performance (kW = kWh / Hours)
    ROUND(AVG(el_kwh / NULLIF(duration_hours, 0)), 2) AS avg_rate_kw,
    -- Count sessions to identify the most active groups
    COUNT(*) AS total_sessions
FROM `VEHICLES.CHARGING_SESSIONS`
WHERE duration_hours > 0 -- Optimising by excluding non-charging events
GROUP BY 
    month_plugin, 
    user_type,
    EXTRACT(MONTH FROM start_plugin) -- Used for chronological sorting
ORDER BY 
    EXTRACT(MONTH FROM start_plugin) ASC, 
    avg_rate_kw DESC;

,month_plugin,user_type,avg_rate_kw,total_sessions
0,Jan,Shared,3.56,247
1,Jan,Private,1.69,1113
2,Feb,Shared,3.14,35
3,Feb,Private,1.53,100
4,Mar,Shared,2.84,41
5,Mar,Private,1.76,260
6,Apr,Shared,3.09,61
7,Apr,Private,1.69,245
8,May,Shared,4.17,69
9,May,Private,1.87,219


Using the charting function, it can be seen that the higher average rate of kWh is higher with Shared Chargers vs. Private Chargers however, changing the series to total_sessions, it can be seen that the private sessions total is always higher than the shared infrastructure, sometimes, in the winter months, substaintially higher.